This is a tutorial for the DFODE-kit package.

In [ ]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

from dfode_kit.df_interface import (
    OneDFreelyPropagatingFlameConfig,
    setup_one_d_flame_case,
    df_to_h5,
)
from dfode_kit.data_operations import (
    touch_h5, 
    get_TPY_from_h5, 
    random_perturb,
    label_npy,
    integrate_h5,
)
from dfode_kit.dfode_core.model.mlp import MLP
from dfode_kit.utils import BCT

DFODE_ROOT = os.environ['DFODE_ROOT']

In [ ]:
from pathlib import Path
import shutil
eq_ratios = [0.9, 1.0, 1.1]

standard_case = 'standard_case_for_sampling'

num_output_steps = 10

standard_case_path = f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/{standard_case}"

for eq_ratio in eq_ratios:
    new_case_name = f"eq_ratio_{eq_ratio}"
    new_case_path = f"{DFODE_ROOT}/tutorials/twoD_HIT_flame/1_sample_train/{new_case_name}"

    if os.path.exists(standard_case_path):
        shutil.copytree(standard_case_path, new_case_path)
        print(f"copy {standard_case} to {new_case_name}")

        original_dir = os.getcwd()
        os.chdir(new_case_path)
        
        try:
            # Operating condition settings
            config_dict = {
                "mechanism": f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
                "T0": 300,
                "p0": 101325,
                "fuel": "H2:1",
                "oxidizer": "O2:0.21,N2:0.79",
                "eq_ratio": eq_ratio,  
            }
            config = OneDFreelyPropagatingFlameConfig(**config_dict)

            # Simulation settings
            settings = {
                "sim_time_step": 1e-6,
                "sim_write_interval": 1e-5,
                "num_output_steps": num_output_steps,
            }
            config.update_config(settings)

            # Setup the case and update dictionary files
            setup_one_d_flame_case(config, '.')
            
            print(f"successfully setting {new_case_name}")
            
        except Exception as e:
            print(f"Wrong in setting {new_case_name}: {e}")
        finally:
            os.chdir(original_dir)
    else:
        print(f"Error")



### A brief introduction to the DFODE method

#### Low-dimensional manifold sampling

A key challenge in preparing training data is achieving sufficient coverage of the relevant thermochemical composition space, which is often prohibitively high-dimensional when detailed chemistry involves tens to hundreds of species. 

To address this, DFODE-kit adopts a low-dimensional
manifold sampling strategy, where thermochemical states are extracted from canonical flame configurations that retain the essential topology of high-dimensional turbulent flames. This approach ensures both computational efficiency and physical representativeness of the training datasets.

In this tutorial, we will demonstrate how to use DFODE-kit to sample a low-dimensional manifold of thermochemical states from a one-dimensional laminar freely propagating flame simulated with DeepFlame. The following code block could also be found in `case_init.ipynb` files within the case templates provides in the `cases` directory. It is used to initialize the simulation and update the dictionary files for the simulation.

Note that at the point, the simulation is not yet started. The user would need to ensure a working version of DeepFlame is available and run the `Allrun` script from command line to start the simulation.

```bash
./Allrun
```

After the simulation is completed, we proceed to use DFODE-kit to gather and manage the thermochemical data.